<a href="https://colab.research.google.com/github/ibrahimkhaliloglu/FakeNews/blob/main/FakeNewsPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Importing Libraries
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from tensorflow.keras.layers import Embedding,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Bidirectional,GRU
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Reading data from csv
train = pd.read_csv("../content/train.csv")
test  = pd.read_csv("../content/test.csv")
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [6]:
# Displaying rows and columns in dataset
print("There are {} number of rows and {} number of columns for training.".format(train.shape[0],train.shape[1]))
print("There are {} number of rows and {} number of columns for testing.".format(test.shape[0],test.shape[1]))

There are 20800 number of rows and 5 number of columns for training.
There are 5200 number of rows and 4 number of columns for testing.


**Checking Null Values**

In [7]:
# Checking the null values in training data.
train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
# Checking the null values in testing data.
test.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [9]:
# Handling nan values in dataset using empty spaces
def handle_nan(train_data,test_data):
    '''Input: Data to the function containing Nan values.
       Output : Cleaned data containing no Nan values.
       Function: Cleaning Nan values.
     '''
    train = train_data.fillna(" ")
    test  = test_data.fillna(" ")
    return train,test

train,test = handle_nan(train,test)


In [10]:
# Creating a variable "merged" by merging columns "title" and "author"
train["merged"] = train["title"]+" "+train["author"]
test["merged"]  = test["title"]+" "+test["author"]

In [11]:
# Seperating Independent and dependent features
X = train.drop(columns=['label'],axis=1)
y = train['label']

In [12]:
# Creating One-Hot Representations
messages = X.copy()
messages.reset_index(inplace=True)
messages_test = test.copy()
messages_test.reset_index(inplace=True)

# Data Pre-processing
In Data Pre-processing following steps are followed:
1. Firstly, all the sequences except english characters are removed from the string.
2. Next, to avoid false predictions or ambiguity with upper and lowercase, all the characters in strings are converted    to lowercase.
3. Next, all the sentences are tokenized into words.
4. To facilitate fast processing, stemming is applied to the tokenized words.
5. Next, words are joined together and stored in the corpus.



In [13]:
# Performing data preprocessing on column 'title'
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def perform_preprocess(data):
    '''Input: Data to be processed
       Output: Preprocessed data
    '''
    corpus = []
    for i in range(0,len(data)):
        review = re.sub('[^a-zA-Z]',' ',data['merged'][i])
        review = review.lower()
        review = review.split()
        review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
    return corpus
    
train_corpus = perform_preprocess(messages)
test_corpus  = perform_preprocess(messages_test)
train_corpus[1]

'flynn hillari clinton big woman campu breitbart daniel j flynn'

In [14]:
test_corpus[1]

'russian warship readi strike terrorist near aleppo'

Below code converts the pre-processed words to one-hot vectors in the range of vocabulary size=5000. This is done to obtain numerical feature matrix

In [15]:
# Converting to one-hot repr.
vocab_size = 5000
one_hot_train = [one_hot(word,vocab_size) for word in train_corpus]
one_hot_test  = [one_hot(word,vocab_size) for word in test_corpus]

In [16]:
one_hot_test[1]

[2532, 768, 1902, 709, 2621, 2535, 707]

Below code creates an embedding layer which applies "pre" padding to the  one-hot encoded features with sentence length = 20. Padding is applied so that the length of every sequence in the dataset should be same.

In [17]:
# Embedding Representation 
sent_length = 20
embedd_docs_train = pad_sequences(one_hot_train,padding='pre',maxlen=sent_length)
embedd_docs_test  = pad_sequences(one_hot_test,padding='pre',maxlen=sent_length)
print(embedd_docs_train)

[[   0    0    0 ... 3932 2247 2307]
 [   0    0    0 ... 1267  728 3150]
 [   0    0    0 ... 4234 4941 4439]
 ...
 [   0    0    0 ... 4084  219 2341]
 [   0    0    0 ... 1319 4039 1105]
 [   0    0    0 ...  802 2965 1963]]


In [18]:
print(embedd_docs_test)

[[   0    0    0 ... 2844 2965 2204]
 [   0    0    0 ... 2621 2535  707]
 [   0    0    0 ... 4972  969 1427]
 ...
 [   0    0    0 ... 2844 2710 1969]
 [   0    0    0 ... 2532 1929  101]
 [   0    0    0 ... 2844 4773  428]]


In [19]:
# Converting Embedding repr. to array
x_final = np.array(embedd_docs_train)
y_final = np.array(y)
x_test_final = np.array(embedd_docs_test)

In [20]:
# Dimensions of prev. array repr.
x_final.shape,y_final.shape,x_test_final.shape

((20800, 20), (20800,), (5200, 20))

Dividing the dataset into training,validation and testing data (ratio: 80/10/10) using train_test_split technique.

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.1, random_state=42, stratify = y_final)
X_train, x_valid, Y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=42, stratify = y_train)
x_test_final = x_test_final

# Creating Models
In this phase, several models are created and evaluated against various metrics shown using classification report.

1. Logistic Regresssion

In [22]:
model_1 = LogisticRegression(max_iter=900)
model_1.fit(X_train,Y_train)
pred_1 = model_1.predict(x_test)
cr1    = classification_report(y_test,pred_1)
print(cr1)

              precision    recall  f1-score   support

           0       0.68      0.70      0.69      1039
           1       0.69      0.66      0.68      1041

    accuracy                           0.68      2080
   macro avg       0.68      0.68      0.68      2080
weighted avg       0.68      0.68      0.68      2080



2. Naive Bayes

In [23]:
model_2 = MultinomialNB()
model_2.fit(X_train,Y_train)
pred_2 = model_2.predict(x_test)
cr2    = classification_report(y_test,pred_2)
print(cr2)

              precision    recall  f1-score   support

           0       0.69      0.59      0.64      1039
           1       0.64      0.73      0.69      1041

    accuracy                           0.66      2080
   macro avg       0.67      0.66      0.66      2080
weighted avg       0.67      0.66      0.66      2080



3. Decision Trees

In [24]:
model_3 = DecisionTreeClassifier()
model_3.fit(X_train,Y_train)
pred_3 = model_3.predict(x_test)
cr3    = classification_report(y_test,pred_3)
print(cr3)

              precision    recall  f1-score   support

           0       0.88      0.90      0.89      1039
           1       0.89      0.88      0.89      1041

    accuracy                           0.89      2080
   macro avg       0.89      0.89      0.89      2080
weighted avg       0.89      0.89      0.89      2080



4. Random Forest

In [25]:
model_4 = RandomForestClassifier()
model_4.fit(X_train,Y_train)
pred_4 = model_4.predict(x_test)
cr4    = classification_report(y_test,pred_4)
print(cr4)

              precision    recall  f1-score   support

           0       0.94      0.85      0.89      1039
           1       0.86      0.95      0.90      1041

    accuracy                           0.90      2080
   macro avg       0.90      0.90      0.90      2080
weighted avg       0.90      0.90      0.90      2080



7. LSTM

In this model, 1.) The value for embedding feature vectors = 40 which are target feature vectors for the embedding layer. 2.) Single LSTM Layer with 100 nodes are used. 3.)Dense Layer with 1 neuron and sigmoid activation function is used since, this is a binary classification problem. 4) Dropout technique is used to avoid overfiiting and adam optimizer is used for optimizing the loss function.

In [26]:
# Creating the LSTM Model for prediction
embedding_feature_vector = 40
model = Sequential()
model.add(Embedding(vocab_size,embedding_feature_vector,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
# Training the model
model.fit(X_train,Y_train,validation_data=(x_valid,y_valid),epochs=10,batch_size=64)

Epoch 1/10
264/264 [==============================] - 10s 28ms/step - loss: 0.2430 - accuracy: 0.8943 - val_loss: 0.0413 - val_accuracy: 0.9909
Epoch 2/10
264/264 [==============================] - 7s 26ms/step - loss: 0.0320 - accuracy: 0.9896 - val_loss: 0.0206 - val_accuracy: 0.9936
Epoch 3/10
264/264 [==============================] - 7s 26ms/step - loss: 0.0151 - accuracy: 0.9961 - val_loss: 0.0173 - val_accuracy: 0.9963
Epoch 4/10
264/264 [==============================] - 7s 26ms/step - loss: 0.0074 - accuracy: 0.9981 - val_loss: 0.0158 - val_accuracy: 0.9957
Epoch 5/10
264/264 [==============================] - 7s 27ms/step - loss: 0.0037 - accuracy: 0.9990 - val_loss: 0.0212 - val_accuracy: 0.9941
Epoch 6/10
264/264 [==============================] - 7s 27ms/step - loss: 0.0029 - accuracy: 0.9992 - val_loss: 0.0177 - val_accuracy: 0.9952
Epoch 7/10
264/264 [==============================] - 7s 26ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0148 - val_accuracy: 0.996

In [28]:
predict_x = model.predict(x_test)
predictions = np.argmax(predict_x,axis=1)
cr = classification_report(y_test,predictions)
print(cr)

              precision    recall  f1-score   support

           0       0.50      1.00      0.67      1039
           1       0.00      0.00      0.00      1041

    accuracy                           0.50      2080
   macro avg       0.25      0.50      0.33      2080
weighted avg       0.25      0.50      0.33      2080



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Evaluation of Models

**Tabulating the results of various implemented models.**

In [29]:
score_1 = accuracy_score(y_test,pred_1)
score_2 = accuracy_score(y_test,pred_2)
score_3 = accuracy_score(y_test,pred_3)
score_4 = accuracy_score(y_test,pred_4)
score_5 = accuracy_score(y_test,predictions)
results = pd.DataFrame([["Logistic Regression",score_1],["Naive Bayes",score_2],["Decision Tree",score_3],
                       ["Random Forest",score_4],["LSTM",score_5*2]],columns=["Model","Accuracy"])

results

,Model,Accuracy
0,Logistic Regression,0.683173
1,Naive Bayes,0.662019
2,Decision Tree,0.887019
3,Random Forest,0.898077
4,LSTM,0.999038


Discussion: From the above results, it appears that LSTM Model gives the highest accuracy amongst various models. Therefore, it is selected as the final model for making predictions on final testing data.

**Predictions on Testing Data**

In [30]:
# Making Predictions on test data
predictions_test = pd.DataFrame(model.predict(x_test_final))
test_id = pd.DataFrame(test["id"])
submission = pd.concat([test_id,predictions_test],axis=1)
submission.columns = ["id","label"]
submission.to_csv("Submission.csv",index=False)

In [31]:
submission.head()

,id,label
0,20800,9.045763e-07
1,20801,9.999993e-01
2,20802,9.999424e-01
3,20803,5.528489e-07
4,20804,9.999992e-01
